## Week 3 Assignment 2B: Toronto Neighborhoods

Applied Data Science Capstone Project  
IBM Data Science Professional Certificate

In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup



- Use BeautifulSoup to scrape URL: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M 





In [2]:
url = urlopen(r"https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(url, 'lxml')
soup = soup.table

- use soup.table data to add each PostalCode(tr.td[0]), Borough(tr.td[1]), Neighborhood(tr.td[2]) to temporary lists
- Create dataframe with columns for PostalCode, Borough & Neighborhood

In [3]:
PostalCode = []
Borough = []
Neighborhood = []

for tr in soup.find_all('tr')[1:]:
    PostalCode.append(tr.find_all('td')[0].get_text().strip('\n'))
    Borough.append(tr.find_all('td')[1].get_text().strip('\n'))
    Neighborhood.append(tr.find_all('td')[2].get_text().strip('\n'))

df = pd.DataFrame({'PostalCode':PostalCode, 'Borough':Borough, 'Neighborhood':Neighborhood})
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


- Iterate through each row and delete rows where 'Borough' value is 'Not assigned' 

NOTE: There are no cases where 'Neighborhood' value is 'Not assigned' when the 'Borough' is assigned.

In [4]:
for index, row in df.iterrows():
    if row['Borough'] == 'Not assigned':
        df.drop(index, inplace=True)

- Reset index and show first 12 rows

In [5]:
df.reset_index(drop=True, inplace=True)
df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [6]:
df.shape

(103, 3)

## Part 2 - Get Geographical Coordinates for Postal Codes

Note: geocoder did not return latitude, longitude coordinates, so I utilized .csv file provided

In [7]:
url = 'https://cocl.us/Geospatial_data'
dfll = pd.read_csv(url)
dfll.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


- Check to ensure same number of rows and columns to prepare for merge

In [8]:
dfll.shape

(103, 3)

- Renamed 'Postal Code' column to 'PostalCode' in csv dataframe to prepare for merge

In [9]:
dfll.rename(columns={"Postal Code":"PostalCode"}, inplace=True)
dfll.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


- Merged two dataframes

In [10]:
dfm = pd.merge(df, dfll, on="PostalCode") 
dfm.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


- Spot checked a few specific 'M5G' PostalCodes from Assigment Instruction Page to ensure merge was correct

In [11]:
dfm.loc[df['PostalCode'] == 'M1R']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
71,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849


In [12]:
dfm.loc[df['PostalCode'] == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


- Final dataframe and shape for Part 2 of this Assignment

In [13]:
dfm.head(11)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [14]:
dfm.shape

(103, 5)